Import packages needed for data processing and plotting

In [2]:
import pandas as pd
import plotly.express as px

Import the subscriptions dataset and convert 'Year' into a datetime format for plotting and further analysis

In [3]:
df = pd.read_excel('data/subscriptions.xlsx')

df['Year'] = pd.to_datetime(df['Year'], format='%Y') 

df

,Year,Netflix,Prime,Disney Plus,Paramount Plus,Max,Hulu
0,2013-01-01,41.43,NaN,NaN,NaN,NaN,NaN
1,2014-01-01,54.48,NaN,NaN,NaN,NaN,NaN
2,2015-01-01,70.84,NaN,NaN,NaN,NaN,NaN
3,2016-01-01,89.08,NaN,NaN,NaN,NaN,NaN
4,2017-01-01,110.64,NaN,NaN,NaN,NaN,NaN
5,2018-01-01,139.26,NaN,NaN,NaN,NaN,20.1
6,2019-01-01,167.09,90.0,NaN,NaN,NaN,23.2
7,2020-01-01,203.66,125.0,33.5,8.0,55.6,28.8
8,2021-01-01,221.84,175.0,68.4,32.8,67.0,37.8
9,2022-01-01,221.64,210.0,87.6,55.9,81.2,41.4


Create chart displaying streaming service subscribers over time

In [53]:
df_long = df.melt(id_vars='Year', var_name='Service', value_name='Subscribers')

# Create the line chart
fig = px.line(df_long, 
              x='Year', 
              y='Subscribers', 
              color='Service', 
              title='<b>Major Streaming Service Subscribers Over Time</b><br><sup>Global Subscribers* (in millions)</sup>',
              template='simple_white',
              markers=True)

#fig.update_traces()
fig.update_layout(yaxis_title='',
                  yaxis = dict(showgrid=True, linecolor='lightgray'),
                  xaxis = dict(showgrid=False, linecolor='lightgray'),
                  font=dict(size=16, family='Trebuchet MS'),
                  title_font=dict(family="Trebuchet MS", size=30,),
                  legend=dict(title_font=dict(weight='bold', size=16)),
                  xaxis_title="")

# Add a annotations
fig.update_layout(
    annotations=[
        # Source info
        dict(x=-0.1, 
             y=-0.15, 
             xref="paper",
             yref="paper",
             showarrow=False, 
             text='Source: searchlogistics.com, businessofapps.com,\nand company reports', 
             font=dict(size=14, color='lightgray', style='italic'),
            #  align='left',
            #  xanchor='left',
             yanchor='bottom'),
        # Notes
        dict(x=1.2,
             y=-0.15,
             xref="paper",
             yref="paper",
             showarrow=False,
             text='*Prime Video numbers are defined by number of users',
             font=dict(size=14, color='lightgray', style='italic'),
            align='right',
            xanchor='right',
            yanchor='bottom'),
        # Watermark
        dict(
            text="@WillsFilms", 
            x=0.3,  
            y=0.3,  
            xref="paper",
            yref="paper",
            showarrow=False,
            font=dict(
                size=50,
                color="lightgrey"
            ),
            align="center",
            valign="middle",
            opacity=0.2,  # Control transparency here
        )
    ])

fig.show()

In [54]:
# Save the figure as a png file
fig.write_image("images/streaming_service_subscribers.png", width=900, height=600)

In [38]:
# Save figure as html file
fig.write_html("html/streaming_service_subscribers.html", include_plotlyjs='cdn', full_html=True)

Create a treemap visual to show the current market share at the last collected data point:

In [63]:
# Filter the most recent year
most_recent_year = df['Year'].max()
df_recent = df[df['Year'] == most_recent_year]

# Melt the dataframe to long format for treemap
df_treemap = df_recent.melt(id_vars='Year', var_name='Service', value_name='Subscribers')

# Create the treemap
fig_treemap = px.treemap(df_treemap, 
                         path=['Service'], 
                         values='Subscribers', 
                         title=f"<b>Market share of the major streamers ({most_recent_year.year})</b><br><sup>Breakdown by proprtion of total</sup>",
                         template='simple_white')

fig_treemap.update_traces(textinfo="label+percent entry")
fig_treemap.update_layout(title_font=dict(family="Trebuchet MS", size=30,),
                            font=dict(size=16, family='Trebuchet MS'),
                            )

# Add a annotations
fig.update_layout(
    annotations=[
        # Source info
        dict(x=3, 
             y=1, 
             xref="paper",
             yref="paper",
             showarrow=False, 
             text='Source: searchlogistics.com, businessofapps.com, and company reports', 
             font=dict(size=14, color='lightgray', style='italic'),
             align='right',
             xanchor='right',
             yanchor='bottom'),
             # Watermark
        dict(
            text="@WillsFilms", 
            x=0.5,  
            y=0.5,  
            xref="paper",
            yref="paper",
            showarrow=False,
            font=dict(
                size=50,
                color="lightgrey"
            ),
            align="center",
            valign="middle",
            opacity=0.2,  # Control transparency here
        )
    ])

fig_treemap.show()

In [ ]:
# Save the treemap as a png file
fig_treemap.write_image("images/streaming_service_market_share.png", width=900, height=600)

In [9]:
# Save figure as html file
fig_treemap.write_html("html/streaming_service_market_share.html", include_plotlyjs='cdn')

Market share over time

In [ ]:
# Calculate market share for each service over time
df_market_share = df.copy()
service_cols = [col for col in df.columns if col != 'Year']
df_market_share['Total'] = df_market_share[service_cols].sum(axis=1, skipna=True)
for col in service_cols:
    df_market_share[col] = df_market_share[col] / df_market_share['Total'] * 100

# Melt to long format for plotting
df_market_share_long = df_market_share.melt(id_vars='Year', value_vars=service_cols, 
                                            var_name='Service', value_name='Market Share (%)')

fig_market_share = px.area(
    df_market_share_long,
    x='Year',
    y='Market Share (%)',
    color='Service',
    title='<b>Streaming Service Market Share Over Time</b><br><sup>Share of Total Subscribers (%)</sup>',
    template='simple_white'
)

fig_market_share.update_layout(
    yaxis_title='Market Share (%)',
    xaxis_title='',
    font=dict(size=16, family='Trebuchet MS'),
    title_font=dict(family="Trebuchet MS", size=30),
    legend=dict(title_font=dict(weight='bold', size=16)),
    xaxis=dict(showgrid=False, linecolor='lightgray'),
    yaxis=dict(showgrid=True, linecolor='lightgray'),
    annotations=[
        # Source info
        dict(
            x=-0.1,
            y=-0.15,
            xref="paper",
            yref="paper",
            showarrow=False,
            text='Source: searchlogistics.com, businessofapps.com,\nand company reports',
            font=dict(size=14, color='lightgray', style='italic'),
            yanchor='bottom'
        ),
        # Watermark
        dict(
            text="@WillsFilms",
            x=0.3,
            y=0.3,
            xref="paper",
            yref="paper",
            showarrow=False,
            font=dict(size=50, color="lightgrey"),
            align="center",
            valign="middle",
            opacity=0.2,
        )
    ]
)

fig_market_share.show()

In [67]:
# save the market share figure as a png file
fig_market_share.write_image("images/streaming_service_market_share_over_time.png", width=900, height=600)

In [68]:
# Save the market share area chart as a html file
fig_market_share.write_html("html/streaming_service_market_share_over_time.html", include_plotlyjs='cdn', full_html=True)

Year-on-year percentage growth

In [ ]:
# Calculate year-on-year percentage growth
df_growth = df.copy()
for service in df.columns[1:]:
    df_growth[service] = df[service].pct_change() * 100

# Melt the dataframe to long format for visualization
df_growth_long = df_growth.melt(id_vars='Year', var_name='Service', value_name='Growth')

,Year,Service,Growth
0,2013-01-01,Netflix,NaN
1,2014-01-01,Netflix,31.498914
2,2015-01-01,Netflix,30.029369
3,2016-01-01,Netflix,25.748165
4,2017-01-01,Netflix,24.202964
...,...,...,...
67,2020-01-01,Hulu,24.137931
68,2021-01-01,Hulu,31.250000
69,2022-01-01,Hulu,9.523810
70,2023-01-01,Hulu,5.555556


In [69]:
# Create a pivot table for the heatmap
growth_pivot = df_growth_long.pivot(index='Service', columns='Year', values='Growth')

fig_heatmap = px.imshow(
    growth_pivot,
    color_continuous_scale='Reds',  # Red shades for positive growth
    aspect='auto',
    labels=dict(color="Growth (%)"),
    title='<b>Year-on-Year Percentage Growth</b><br><sup>Major Streaming Services</sup>',
)

fig_heatmap.update_layout(
    font=dict(size=16, family='Trebuchet MS'),
    title_font=dict(family="Trebuchet MS", size=30),
    xaxis_title="",
    yaxis_title="",
    plot_bgcolor='rgba(0,0,0,0)',  # Transparent background
    coloraxis_colorbar=dict(title="Growth (%)"),
    yaxis = dict(
        showgrid=False,
        linecolor='lightgray',
        tickmode='array',  # This ensures all ticks are displayed
        ticks='outside',  
        ticklen=8,  # Optional: controls the length of the ticks
        tickcolor='white',  
        tickprefix='    ',  # Add space before labels to create padding
    ),
    xaxis = dict(
        showgrid=False,
        linecolor='lightgray',
        tickmode='array',  # Add ticks to provide padding
        ticks='outside', 
        ticklen=8,  # Optional: controls the length of the ticks
        tickcolor='white',  # Hide ticks
    ),
)

# Add annotations
fig.update_layout(
    annotations=[
        dict(
            x=-0.1,
            y=-0.15,
            xref="paper",
            yref="paper",
            showarrow=False,
            text='Source: searchlogistics.com, businessofapps.com,\nand company reports',
            font=dict(size=14, color='lightgray', style='italic'),
            yanchor='bottom'
        ),
        # Watermark
        dict(
            text="@WillsFilms",
            x=0.3,
            y=0.3,
            xref="paper",
            yref="paper",
            showarrow=False,
            font=dict(size=50, color="lightgrey"),
            align="center",
            valign="middle",
            opacity=0.2,
        )
    ]
)

fig_heatmap.show()

In [70]:
# Save the treemap as a png file
fig_heatmap.write_image("images/streaming_service_yearly_growth.png", width=900, height=600)

In [71]:
# Save the growth figure as html file
fig_heatmap.write_html("html/streaming_service_yearly_growth.html", include_plotlyjs='cdn')